In [5]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
from datetime import datetime, time, timedelta
import time
import re
import json
import random
import os

In [2]:
# Запрос веб-страницы
url = 'http://books.toscrape.com/'
response = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'})

# Парсинг HTML-содержимого веб-страницы с помощью Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
print(response)
#print(soup)

<Response [200]>


In [5]:
# Вывод ссылок на
release_links = []
while True:

    for link in soup.find_all('li', ('class', 'col-xs-6 col-sm-4 col-md-3 col-lg-3')):
        try:
            rr = link.find('a').get('href')
        except:
            rr = None
        finally:
            if rr != None:
                rr = rr if not rr.find('catalogue') else ''.join(['catalogue/',rr])
                release_links.append(rr)

    next_button = soup.find('li', ('class', 'next'))
    try:
        next_link = next_button.find('a').get('href')
    except:
        print('Последняя страница')
        break
    next_link = next_link if not next_link.find('catalogue') else ''.join(['catalogue/',next_link])
    if next_link:
        print(f'Получение страницы: {urllib.parse.urljoin(url, next_link)}')
        time.sleep(0.5)
        response = requests.get(urllib.parse.urljoin(url, next_link), headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'})
        soup = BeautifulSoup(response.content, 'html.parser')
        #break
    else:
        print('no next')
        break

print(f'Найдено ссылок: {len(release_links)}')
#print(release_links)

Получение страницы: http://books.toscrape.com/catalogue/page-2.html
Получение страницы: http://books.toscrape.com/catalogue/page-3.html
Получение страницы: http://books.toscrape.com/catalogue/page-4.html
Получение страницы: http://books.toscrape.com/catalogue/page-5.html
Получение страницы: http://books.toscrape.com/catalogue/page-6.html
Получение страницы: http://books.toscrape.com/catalogue/page-7.html
Получение страницы: http://books.toscrape.com/catalogue/page-8.html
Получение страницы: http://books.toscrape.com/catalogue/page-9.html
Получение страницы: http://books.toscrape.com/catalogue/page-10.html
Получение страницы: http://books.toscrape.com/catalogue/page-11.html
Получение страницы: http://books.toscrape.com/catalogue/page-12.html
Получение страницы: http://books.toscrape.com/catalogue/page-13.html
Получение страницы: http://books.toscrape.com/catalogue/page-14.html
Получение страницы: http://books.toscrape.com/catalogue/page-15.html
Получение страницы: http://books.toscrape.

In [6]:
# Объединение ссылок с базовым URL-адресом для создания списка URL-адресов
url_joined = []
for link in release_links:
  url_joined.append(urllib.parse.urljoin(url, link))

In [7]:
# Сохранить данные в файле
# на всякий случай

with open('books_links_raw.json', 'w') as f:
    json.dump(release_links, f)

with open('books_links.json', 'w') as f:
    json.dump(url_joined, f)

print(len(url_joined))

1000


In [6]:
# Считать данные из файла
# для экономии времени

url_joined = []
with open('books_links.json', 'r') as f:
    url_joined = json.load(f)

print(len(url_joined))

1000


In [7]:

# сохранение данных в JSON-файл
def save_data(data):
    with open('./books.json', 'w') as f:
        json.dump(data, f)
    print(f'Сохранено в файл {len(data)} записей')


# Извлечение данных из таблицы построково и сохранение их в списке словарей
def get_from_url(data, url_joined):
    for i, url in enumerate(url_joined):
        response = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'})
        soup = BeautifulSoup(response.content, 'html.parser')

        row_data={}
        try:
            name = soup.find('div', {'class': 'col-sm-6 product_main'}).find('h1').text.strip()
        except:
            continue
        row_data['Name'] = name

        table = soup.find('table', {'class': 'table table-striped'})
        rows = table.find_all('tr')
        for row in rows:
            key = row.find('th').text.strip()
            value = row.find('td').text.strip()
            #print(key, value)
            if key == 'Price (incl. tax)':
                value = value
                row_data[key] = value
            elif key == 'Availability':
                value = int(re.sub('[^0-9]', '', value))
                row_data[key] = value
    
        article = soup.find('article', {'class': 'product_page'})
        paragraph = article.find_all('p')[3].text.strip()
        row_data['Description'] = paragraph
        data.append(row_data)

        print(name)
        save_data(data)
        #time.sleep(random.randint(1,5))


In [8]:
# глюки какие-то лезут, программа зависает на случайной строке и висит бесконечно
# посему сохраняем каждый добытый элемент и при перезапуске программы продолжаем с прошлого сохраненного места

data = []

if os.path.exists('./books.json'):
    with open('./books.json', 'r') as f:
        data = json.load(f)

print(f'Вытаскиваем элементы с {len(data)} по {len(url_joined)}')
get_from_url(data, url_joined[len(data):])

print(f'Загрузка завершена. Всего в файле сохранено {len(data)} элементов')


Вытаскиваем элементы с 600 по 1000
The Dream Thieves (The Raven Cycle #2)
Сохранено в файл 601 записей
The Darkest Corners
Сохранено в файл 602 записей
The Crossover
Сохранено в файл 603 записей
The 5th Wave (The 5th Wave #1)
Сохранено в файл 604 записей
Tell the Wind and Fire
Сохранено в файл 605 записей
Tell Me Three Things
Сохранено в файл 606 записей
Talking to Girls About Duran Duran: One Young Man's Quest for True Love and a Cooler Haircut
Сохранено в файл 607 записей
Siddhartha
Сохранено в файл 608 записей
Shiver (The Wolves of Mercy Falls #1)
Сохранено в файл 609 записей
Remember Me?
Сохранено в файл 610 записей
Red Dragon (Hannibal Lecter #1)
Сохранено в файл 611 записей
Peak: Secrets from the New Science of Expertise
Сохранено в файл 612 записей
My Mother Was Nuts
Сохранено в файл 613 записей
Mexican Today: New and Rediscovered Recipes for Contemporary Kitchens
Сохранено в файл 614 записей
Maybe Something Beautiful: How Art Transformed a Neighborhood
Сохранено в файл 615 запи